In [14]:
import tensorflow as tf
from tensorflow.keras import Model, layers
import sinkhorn as sh
import numpy as np
from utils import *
inv = InvarianceNNGraph()

In [15]:
y0 = np.random.binomial(1, 0.5, (1200,))
y1 = np.random.binomial(1, 0.8, (1500,))
f = lambda y: np.random.normal(1, 1, (64,)) if y else np.random.normal(0, 1, (64,))
x0 = [f(y) for y in y0]
x1 = [f(y) for y in y1]
y0 = tf.one_hot(y0, 2)
y1 = tf.one_hot(y1, 2)
x0 = tf.cast(x0, dtype = tf.float32)
x1 = tf.cast(x1, dtype = tf.float32)
data = ((x0, y0), (x1, y1))

In [16]:
batch_size = 150
num_steps = 100
batch_data = []
for env in data:
    batch = tf.data.Dataset.from_tensor_slices((env[0], env[1]))
    batch = batch.repeat().shuffle(5000).batch(batch_size).prefetch(1)
    batch_data.append(batch.take(num_steps))
    
learningRate = 0.01
optimizer = tf.optimizers.Adam(learningRate)


In [17]:
for step, data in enumerate(zip(*batch_data), 1):
    with tf.GradientTape() as g:
        loss = tf.cast(0, dtype = tf.float32)
        for index, (x, y) in enumerate(data):
            probs = inv(x, env = index)
            loss = loss + EntropyLoss(y, probs)
        for label in [0,1]:
            conditional_data = [env[0][env[1][:, 1] == label] for env in data]
            loss = loss + sh.sinkhorn_dist(conditional_data[0], conditional_data[1], 2, 5)
        variables = inv.trainable_variables
        for v in variables:
            loss = loss + tf.norm(v)
    trainable_variables = inv.trainable_variables
    gradients = g.gradient(loss, trainable_variables)
    optimizer.apply_gradients(zip(gradients, trainable_variables))

In [18]:
inv.trainable_variables

[<tf.Variable 'Variable:0' shape=(64, 32) dtype=float32, numpy=
 array([[-9.92676214e-05, -1.57316739e-04, -1.78686634e-04, ...,
         -7.10881141e-04,  2.53825565e-04, -9.39139136e-05],
        [-2.71522847e-04, -8.93743709e-05, -1.77936570e-04, ...,
          3.07079754e-04,  4.12645808e-04,  8.31990619e-05],
        [-1.48085834e-04,  4.04660823e-05, -8.55748949e-05, ...,
          8.35271669e-04,  3.42591229e-04, -1.95558459e-05],
        ...,
        [-2.37094355e-07,  3.68921203e-04,  6.62091654e-04, ...,
          2.05228687e-04,  1.00969366e-04, -7.26397848e-05],
        [-1.16517069e-04,  6.14211895e-05,  1.76137692e-04, ...,
         -7.79293769e-04,  8.19943496e-04, -7.99942645e-05],
        [ 1.94504741e-04, -8.76174308e-05, -2.82099994e-04, ...,
          2.30301521e-04,  7.05024169e-04, -9.07807669e-04]], dtype=float32)>,
 <tf.Variable 'Variable:0' shape=(32, 16) dtype=float32, numpy=
 array([[-2.22895760e-04,  6.73069444e-05, -4.12846566e-05,
          3.32894793e-04,